In [4]:
import os
from PIL import Image
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
from typing import Tuple
import numpy as np

# Локальна порогова обрабка

### Метод Niblack

Метод використовується практично для швидкої фільтрації контрастних зображень, у яких відсутні сильно зашумлені області з плавними переходами яскравості.

- μ(x, y) – середнє і s(x, y) – середньоквадратичне відхилення вибірки для деякої околиці точки.
- Значення k визначає, яку частину межі об'єкта взяти як самий об'єкт.

In [2]:
def niblack(image, k, size=5):

    height, width = image.shape
    image_copy = image.copy()

    radius = (size - 1) // 2

    for i in range(radius, height - radius):
        
        for j in range(radius, width - radius):

            block = image[i - radius:i + radius + 1, j - radius:j + radius + 1]

            mean_value = np.sum(block) / (size * size)
            block_std = np.sqrt(np.sum((block - mean_value) ** 2) / (size * size))
        
            threshold = mean_value + k * block_std
            
            if image[i, j] < threshold:
                image_copy[i, j] = 0
            else:
                image_copy[i, j] = 255

    return image_copy

### Саувола метод

- k - коефіцієнт контролю в діапазоні [0,2, 0,5] 
- R – це попередньо визначене значення рівня сірого зображення.
- s - стандартне відхилення околиці

$$
T(x, y) = \mu(x, y) \cdot \left[ 1 - k \cdot \left(1 - \frac{s(x, y)}{R} \right) \right]
$$

In [3]:
def sauvola(image, size=5, max_value=128, k=0.5):
    
    height, width = image.shape
    output_image = np.copy(image)

    radius = (size - 1) // 2

    for i in range(radius, height - radius):
        for j in range(radius, width - radius):

            block = image[i - radius:i + radius + 1, j - radius:j + radius + 1]

            block_mean = np.mean(block)
            block_std = np.std(block)

            threshold = block_mean * (1 - k * (1 - (block_std / max_value)))
            if image[i, j] < threshold:
                output_image[i, j] = 0
            else:
                output_image[i, j] = 255

    return output_image

### Крістіан метод

In [5]:
def christian(image, size=5, k=0.5):

    height, width = image.shape
    output_image = np.copy(image)

    min_value = np.min(image)
    max_block_std = 0  
    
    for x in range(size // 2, height - size // 2):
        for y in range(size // 2, width - size // 2):

            block = image[x - (size // 2): x + (size // 2) + 1, y - (size // 2): y + (size // 2) + 1]

            block_std = np.std(block)
   
            if block_std > max_block_std:
                max_block_std = block_std

    radius = (size - 1) // 2

    for i in range(radius, height - radius):
        for j in range(radius, width - radius):

            block = image[i - radius:i + radius + 1, j - radius:j + radius + 1]

            block_mean = np.mean(block)
            block_std = np.std(block)

            threshold = (1 - k) * block_mean + k * min_value + k * (block_std / max_block_std) * (block_mean - min_value)
 
            if image[i, j] < threshold:
                output_image[i, j] = 0
            else:
                output_image[i, j] = 255

    return output_image

In [6]:
filepath = "/Users/yurab/Desktop/IgromanYTS-basics-of-computer-vision/Lab1/image.png"

In [7]:
def resize_with_padding(image: np.array, new_shape: Tuple[int, int], padding_color: Tuple[int] = (255, 255, 255)) -> np.array:
    original_shape = (image.shape[1], image.shape[0])
    scale_ratio = float(max(new_shape)) / max(original_shape)
    new_size = tuple([int(dim * scale_ratio) for dim in original_shape])

    if new_size[0] > new_shape[0] or new_size[1] > new_shape[1]:
        scale_ratio = float(min(new_shape)) / min(original_shape)
        new_size = tuple([int(dim * scale_ratio) for dim in original_shape])

    image = cv2.resize(image, new_size)
    delta_rows = new_shape[0] - new_size[0]
    delta_cols = new_shape[1] - new_size[1]
    top, bottom = delta_cols // 2, delta_cols - (delta_cols // 2)
    left, right = delta_rows // 2, delta_rows - (delta_rows // 2)

    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, None, value=padding_color)
    return image

In [10]:
img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)

niblack_img = niblack(img,-0.2)
cv2.imwrite('image_n.png', niblack_img)

sauvola_img= sauvola(img)
cv2.imwrite('image_s.png', sauvola_img)

christian_img = christian(img, )
cv2.imwrite('image_c.png', christian_img)

True